In [13]:
import pickle
import time

from sentence_transformers import SentenceTransformer

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

from plotly.io import to_html

import plotly.express as px
import pandas as pd
import umap.umap_ as umap
# from umap import plot
# import umap.umap_.plot as umap
import numpy as np
import time
import itertools

In [9]:
# model = "microsoft/Multilingual-MiniLM-L12-H384"
# model = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# model = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"
model = "DeepPavlov/rubert-base-cased-sentence"
# model = "DeepPavlov/distilrubert-base-cased-conversational"
# model = "sentence-transformers/distiluse-base-multilingual-cased"

In [10]:
companies = pd.read_csv('../data/companies_large.csv', index_col=0)
companies.sample(5)

,Название,Описание,Источник,Brand-Image,Отрасль,Категория,Подкатегория,ИНН,ОГРН,КПП,Адрес,Телефоны,Почта,Сайт,Вконтакте,Год основания,Количество сотрудников,Название юридического лица,Дата регистрации,YouTube
4793,Росэлектрод-Долина,Основная деятельность ООО «РОСЭлектрод - Долин...,http://manufacturers.ru/company/roselektrod-do...,http://manufacturers.ru/sites/default/files/st...,Металлургия,Металлообработка,Электродная промышленность,5614080705,1175658024783,561401001.0,"город Орск, Тобольская улица, дом 8",+7 (3537) 26-90-52 (Отдел сбыта) +7 (3537) 26-...,roselectrod_dol@mail.ru (основная),http://roselektrod-dolina.ru,NaN,NaN,1,"Росэлектрод-Долина, ООО",30.11.2017,NaN
8935,Инбуко,Наша фирма работает на рынке пищевой промышлен...,http://manufacturers.ru/company/inbuko,http://manufacturers.ru/sites/default/files/st...,Пищевая промышленность,Прочее пищевое производство,Производство пищевых добавок,2311072611,1032306445698,231101001.0,"Краснодар, ул. Ростовское шоссе, д. 76",(861) 279-00-60 8 (903) 454-47-74,NaN,http://inbuco.net,NaN,NaN,7,"Инбуко, ООО",07.08.2003,NaN
9386,Геополитекс,«Геополитекс» — это современное и быстро разви...,http://manufacturers.ru/company/geopoliteks,http://manufacturers.ru/sites/default/files/st...,"Легкая промышленность, Химическая промышленность","Текстильная промышленность, Органическое (нефт...","Производство геотекстильных, нетканых материал...",5258147631,1205200006330,525801001.0,"г. Москва, Раменский г. о., р.п. Быково, ул. А...",8 (800) 500-28-01 (по России) 8 (499) 444-16-5...,msk@geopolitex.ru (Москва),http://geopolitex.ru,NaN,NaN,NaN,"Геополитекс, ООО",04.02.2020,NaN
8407,Броксталь,ООО «Броксталь» производит полный комплекс обо...,http://manufacturers.ru/company/brokstal-0,http://manufacturers.ru/sites/default/files/st...,Металлургия,Металлообработка,Производство металлоизделий,1215074741,1021200777795,120701001.0,"г. Киров, ул. Калинина, д. 38, оф. 227",8-800-500-07-82,barinov@broksteel.ru,http://kirov.broksteel.ru,NaN,NaN,154,"Броксталь, ООО",23.12.2002,NaN
5840,Озерное,Агрофирма “Озерное” располагает полностью уком...,http://manufacturers.ru/company/ozernoe,http://manufacturers.ru/sites/default/files/st...,Сельское хозяйство,Растениеводство,Овощеводство,6001004390,1126031000017,600101001.0,д. Махново,+7 (81141) 2-14-72 +7 (911) 884-29-66,ozernoe.info@gmail.com,http://ozernoe.info,NaN,NaN,37,"Озерное, ООО",10.01.2012,NaN


In [11]:
sentence_embedder = SentenceTransformer(model, device='cuda:0')
sentence_embedder.max_seq_length = 512
sentence_embedder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [12]:
# sentence_embedder = sentence_embedder.cpu()

In [13]:
# def chunks(iterable, n):
#     return [iterable[i:i+n] for i in range(0, len(iterable), n)]

In [14]:
descriptions_embs = sentence_embedder.encode(list(companies['Описание']), batch_size=32, show_progress_bar=True)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.80 GiB total capacity; 3.92 GiB already allocated; 38.44 MiB free; 3.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# with open('../data/description_embeddings.npy','rb') as f:
#     pickle.dump(descriptions_embs, f)

In [5]:
with open('../data/description_embeddings.npy','rb') as f:
    descriptions_embs = pickle.load(f)
descriptions_embs.shape

(12600, 768)

In [6]:
descriptions_embs = descriptions_embs.astype(np.float16)
descriptions_embs.shape

(12600, 768)

In [42]:
umap_3d = umap.UMAP(n_components=3, n_neighbors=10, min_dist=0.7, init='random')

In [43]:
proj_3d = umap_3d.fit_transform(descriptions_embs)

In [44]:
categories = companies['Отрасль'].fillna('Null').apply(lambda x: x.split(',')[0])

In [46]:
fig = px.scatter_3d(x=proj_3d[:, 0], y=proj_3d[:, 1], z=proj_3d[:, 2], hover_name=companies['Название'],
                    template='plotly_white', width=1400, height=800, color=categories)
fig.update_layout(showlegend=False)
fig.show()

In [47]:
html_code = to_html(fig, default_width="400px", default_height="400px")
with open("../data/plot_full.html", "w") as text_file:
    text_file.write(html_code)